In [2]:
# Core deep learning and data processing
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd

# Training utilities
import random
import os
import math
from tqdm import tqdm

In [ ]:
# Visualization tools
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.font_manager as fm

# Experiment tracking
import wandb

# Additional PyTorch utilities
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

In [3]:
# Configure computation device
def setup_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("Using GPU for computation")
    else:
        device = torch.device("cpu")
        print("Using CPU for computation")
    return device

# Initialize device
device = setup_device()

device(type='cuda')

In [4]:
wandb.login(key= '1b5f670bdb4b8ed39a9bc34744dd738c9b33dede')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs24m022 (cs24m022-iit-madras-foundation) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Get Data

In [5]:
# Define data paths
def get_data_paths():
    base_path = "/kaggle/input/dakshina22/hi/lexicons"
    return {
        'train': f"{base_path}/hi.translit.sampled.train.tsv",
        'dev': f"{base_path}/hi.translit.sampled.dev.tsv",
        'test': f"{base_path}/hi.translit.sampled.test.tsv"
    }

# Initialize data paths
data_paths = get_data_paths()
train_path = data_paths['train']
dev_path = data_paths['dev']
test_path = data_paths['test']

# Display paths
print("Data paths configured:")
for key, path in data_paths.items():
    print(f"{key.capitalize()} data: {path}")

## Data Loading and Preprocessing

In [6]:
def load_datasets(paths):
    """
    Load and preprocess datasets from TSV files
    """
    # Define column names
    columns = ['devanagari', 'latin', 'frequency']
    
    # Load datasets with error handling
    try:
        datasets = {
            'train': pd.read_csv(paths['train'], sep='\t', header=None, names=columns),
            'dev': pd.read_csv(paths['dev'], sep='\t', header=None, names=columns),
            'test': pd.read_csv(paths['test'], sep='\t', header=None, names=columns)
        }
        
        # Print dataset statistics
        print("Dataset Statistics:")
        print("-" * 50)
        for name, df in datasets.items():
            print(f"{name.capitalize()} set: {len(df):,} examples")
            print(f"Columns: {', '.join(df.columns)}")
            print(f"Memory usage: {df.memory_usage().sum() / 1024**2:.2f} MB")
            print("-" * 50)
            
        return datasets
        
    except Exception as e:
        print(f"Error loading datasets: {str(e)}")
        return None

# Load all datasets
datasets = load_datasets(data_paths)
train_df = datasets['train']
dev_df = datasets['dev']
test_df = datasets['test']

Train Dataset Size : 44204
Dev Dataset Size   : 4358
Test Dataset Size  : 4502


In [8]:
class SequenceProcessor:
    def __init__(self):
        # Define control tokens with unique identifiers
        self.control_tokens = {
            'boundary_start': '<BOS>',  # Beginning of sequence
            'boundary_end': '<EOS>',    # End of sequence
            'padding': '<PAD>',         # Padding token
            'unknown': '<UNK>'          # Unknown token
        }
        
        # Initialize token registry
        self.registry = {
            self.control_tokens['boundary_start']: 0,
            self.control_tokens['boundary_end']: 1,
            self.control_tokens['padding']: 2,
            self.control_tokens['unknown']: 3
        }
        
        # Create reverse lookup
        self.reverse_registry = {v: k for k, v in self.registry.items()}
        
        # Track total tokens
        self.total_tokens = len(self.registry)
    
    def register_tokens(self, text_sequences):
        """
        Register new tokens from input sequences
        """
        for sequence in text_sequences:
            sequence = str(sequence)
            for token in sequence:
                if token not in self.registry:
                    self.registry[token] = self.total_tokens
                    self.reverse_registry[self.total_tokens] = token
                    self.total_tokens += 1
    
    def sequence_to_indices(self, sequence, wrap_boundaries=True):
        """
        Convert text sequence to numerical indices
        """
        sequence = str(sequence)
        indices = []
        
        # Convert each token to its index
        for token in sequence:
            index = self.registry.get(token, self.registry[self.control_tokens['unknown']])
            indices.append(index)
        
        # Add boundary tokens if requested
        if wrap_boundaries:
            indices = ([self.registry[self.control_tokens['boundary_start']]] + 
                      indices + 
                      [self.registry[self.control_tokens['boundary_end']]])
        
        return indices
    
    def indices_to_sequence(self, indices, strip_boundaries=True):
        """
        Convert numerical indices back to text sequence
        """
        sequence = []
        
        for index in indices:
            # Handle tensor indices
            if isinstance(index, torch.Tensor):
                index = index.item()
            
            # Get token from index
            if index in self.reverse_registry:
                token = self.reverse_registry[index]
                
                # Skip control tokens if requested
                if strip_boundaries and token in self.control_tokens.values():
                    continue
                    
                sequence.append(token)
        
        return ''.join(sequence)
    
    def get_control_index(self, control_type):
        """
        Get index for a specific control token
        """
        return self.registry[self.control_tokens[control_type]]
    
    def get_sequence_length(self, sequence):
        """
        Get length of sequence including boundary tokens
        """
        return len(self.sequence_to_indices(sequence, wrap_boundaries=True))
    
    def is_control_token(self, token):
        """
        Check if a token is a control token
        """
        return token in self.control_tokens.values()

In [9]:
class BilingualPairDataset(Dataset):
    def __init__(self, file_path, input_processor, output_processor):
        """
        Initialize dataset for bilingual text pairs
        """
        # Load and preprocess data
        self.data = pd.read_csv(file_path, sep='\t', header=None, 
                              names=['output', 'input', 'metadata'])
        
        # Initialize storage
        self.pairs = []
        self._process_pairs(input_processor, output_processor)
        
        # Track dataset statistics
        self.stats = {
            'total_pairs': 0,
            'max_input_len': 0,
            'max_output_len': 0,
            'avg_input_len': 0,
            'avg_output_len': 0
        }
        self._compute_statistics()
    
    def _process_pairs(self, input_processor, output_processor):
        """
        Process input-output pairs
        """
        for _, row in self.data.iterrows():
            # Process input sequence
            input_indices = input_processor.sequence_to_indices(row['input'])
            
            # Process output sequence
            output_indices = output_processor.sequence_to_indices(row['output'])
            
            # Store processed pair
            self.pairs.append({
                'input': input_indices,
                'output': output_indices,
                'input_len': len(input_indices),
                'output_len': len(output_indices)
            })
    
    def _compute_statistics(self):
        """
        Compute dataset statistics
        """
        if not self.pairs:
            return
            
        self.stats['total_pairs'] = len(self.pairs)
        self.stats['max_input_len'] = max(pair['input_len'] for pair in self.pairs)
        self.stats['max_output_len'] = max(pair['output_len'] for pair in self.pairs)
        self.stats['avg_input_len'] = sum(pair['input_len'] for pair in self.pairs) / self.stats['total_pairs']
        self.stats['avg_output_len'] = sum(pair['output_len'] for pair in self.pairs) / self.stats['total_pairs']
    
    def __len__(self):
        """
        Return total number of pairs
        """
        return self.stats['total_pairs']
    
    def __getitem__(self, idx):
        """
        Get a specific pair by index
        """
        pair = self.pairs[idx]
        return (
            torch.tensor(pair['input'], dtype=torch.long),
            torch.tensor(pair['output'], dtype=torch.long)
        )
    
    def get_statistics(self):
        """
        Return dataset statistics
        """
        return self.stats
    
    def get_pair_lengths(self, idx):
        """
        Get lengths of input and output sequences for a pair
        """
        pair = self.pairs[idx]
        return pair['input_len'], pair['output_len']
    
    def get_max_lengths(self):
        """
        Get maximum lengths of input and output sequences
        """
        return self.stats['max_input_len'], self.stats['max_output_len']

In [10]:
def prepare_batch(batch_data):
    """
    Prepare and pad batch data for model input
    """
    # Separate input and output sequences
    input_sequences = [pair[0] for pair in batch_data]
    output_sequences = [pair[1] for pair in batch_data]
    
    # Calculate padding lengths
    max_input_len = max(len(seq) for seq in input_sequences)
    max_output_len = max(len(seq) for seq in output_sequences)
    
    # Pad sequences
    padded_inputs = torch.zeros(len(input_sequences), max_input_len, dtype=torch.long)
    padded_outputs = torch.zeros(len(output_sequences), max_output_len, dtype=torch.long)
    
    # Fill padded tensors
    for i, (in_seq, out_seq) in enumerate(zip(input_sequences, output_sequences)):
        padded_inputs[i, :len(in_seq)] = torch.tensor(in_seq)
        padded_outputs[i, :len(out_seq)] = torch.tensor(out_seq)
    
    return padded_inputs, padded_outputs

In [11]:
def initialize_processors(train_data):
    """
    Initialize and build token processors from training data
    """
    # Create processors
    input_processor = SequenceProcessor()
    output_processor = SequenceProcessor()
    
    # Extract sequences
    input_sequences = train_data['input'].tolist()
    output_sequences = train_data['output'].tolist()
    
    # Build token mappings
    input_processor.register_tokens(input_sequences)
    output_processor.register_tokens(output_sequences)
    
    # Print statistics
    print("Input Processor Statistics:")
    print(f"Total tokens: {input_processor.total_tokens}")
    print(f"Control tokens: {len(input_processor.control_tokens)}")
    print("\nOutput Processor Statistics:")
    print(f"Total tokens: {output_processor.total_tokens}")
    print(f"Control tokens: {len(output_processor.control_tokens)}")
    
    return input_processor, output_processor

In [12]:
def create_data_loaders(input_processor, output_processor, batch_size=32):
    """
    Create and configure data loaders for all datasets
    """
    # Create datasets
    datasets = {
        'train': BilingualPairDataset(train_path, input_processor, output_processor),
        'dev': BilingualPairDataset(dev_path, input_processor, output_processor),
        'test': BilingualPairDataset(test_path, input_processor, output_processor)
    }
    
    # Create data loaders
    loaders = {
        'train': DataLoader(
            datasets['train'],
            batch_size=batch_size,
            shuffle=True,
            collate_fn=prepare_batch
        ),
        'dev': DataLoader(
            datasets['dev'],
            batch_size=batch_size,
            shuffle=False,
            collate_fn=prepare_batch
        ),
        'test': DataLoader(
            datasets['test'],
            batch_size=batch_size,
            shuffle=False,
            collate_fn=prepare_batch
        )
    }
    
    return datasets, loaders

In [13]:
def verify_data_loader(loader, input_processor, output_processor):
    """
    Verify data loader configuration and sample data
    """
    # Get a batch
    for batch in loader:
        inputs, outputs = batch
        
        # Print shapes
        print("Batch Configuration:")
        print(f"Input shape: {inputs.shape}")
        print(f"Output shape: {outputs.shape}")
        
        # Process first sample
        sample_input = inputs[0]
        sample_output = outputs[0]
        
        # Convert to text
        input_text = input_processor.indices_to_sequence(sample_input)
        output_text = output_processor.indices_to_sequence(sample_output)
        
        print("\nSample Translation:")
        print(f"Input:  {input_text}")
        print(f"Output: {output_text}")
        
        # Print sequence lengths
        print("\nSequence Lengths:")
        print(f"Input length:  {len(input_text)}")
        print(f"Output length: {len(output_text)}")
        
        break

Inputs shape: torch.Size([32, 14])
Targets shape: torch.Size([32, 13])
Sample input: tensor([ 1,  5,  8, 17,  4,  4,  5,  2,  0,  0,  0,  0,  0,  0])
Sample target: tensor([ 1, 16,  9, 15, 21, 16,  2,  0,  0,  0,  0,  0,  0])
nidaan
निदान


## With Out Attention (Vanilla) Model

In [15]:
class SequenceEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers=1, rnn_type="gru", drop_rate=0.0):
        super().__init__()
        
        # Configuration
        self.config = {
            'vocab_size': vocab_size,
            'embed_dim': embed_dim,
            'hidden_dim': hidden_dim,
            'num_layers': num_layers,
            'rnn_type': rnn_type.lower(),
            'drop_rate': drop_rate if num_layers > 1 else 0.0
        }
        
        # Token embedding layer with layer normalization
        self.token_embedding = nn.Sequential(
            nn.Embedding(vocab_size, embed_dim),
            nn.LayerNorm(embed_dim)
        )
        
        # RNN layer selection and initialization
        self._init_rnn_layer()
        
        # Dropout for regularization
        self.dropout = nn.Dropout(self.config['drop_rate'])
        
        # Output projection layer
        self.output_projection = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(self.config['drop_rate']),
            nn.LayerNorm(hidden_dim)
        )
    
    def _init_rnn_layer(self):
        """
        Initialize the appropriate RNN layer based on configuration
        """
        rnn_config = {
            'input_size': self.config['embed_dim'],
            'hidden_size': self.config['hidden_dim'],
            'num_layers': self.config['num_layers'],
            'dropout': self.config['drop_rate'],
            'batch_first': True
        }
        
        if self.config['rnn_type'] == 'lstm':
            self.rnn = nn.LSTM(**rnn_config)
        elif self.config['rnn_type'] == 'gru':
            self.rnn = nn.GRU(**rnn_config)
        else:
            self.rnn = nn.RNN(**rnn_config)
    
    def forward(self, x):
        """
        Process input sequence through the encoder
        """
        # Get sequence lengths for packing
        lengths = (x != 0).sum(dim=1)
        
        # Embed tokens
        embedded = self.token_embedding(x)
        embedded = self.dropout(embedded)
        
        # Pack sequences for efficiency
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        
        # Process through RNN
        if self.config['rnn_type'] == 'lstm':
            packed_outputs, (hidden, cell) = self.rnn(packed_embedded)
            outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs, batch_first=True)
            return self.output_projection(outputs), (hidden, cell)
        else:
            packed_outputs, hidden = self.rnn(packed_embedded)
            outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs, batch_first=True)
            return self.output_projection(outputs), hidden
    
    def get_config(self):
        """
        Return encoder configuration
        """
        return self.config.copy()
    
    def get_output_dim(self):
        """
        Return output dimension
        """
        return self.config['hidden_dim']

In [16]:
class SequenceDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers=1, rnn_type="gru", drop_rate=0.0):
        super().__init__()
        
        # Store configuration
        self.settings = {
            'vocab_size': vocab_size,
            'embed_dim': embed_dim,
            'hidden_dim': hidden_dim,
            'num_layers': num_layers,
            'rnn_type': rnn_type.lower(),
            'drop_rate': drop_rate if num_layers > 1 else 0.0
        }
        
        # Token embedding with normalization
        self.token_embedding = nn.Sequential(
            nn.Embedding(vocab_size, embed_dim),
            nn.LayerNorm(embed_dim)
        )
        
        # Initialize RNN layer
        self._setup_rnn()
        
        # Regularization
        self.dropout = nn.Dropout(self.settings['drop_rate'])
        
        # Output layers
        self.output_layers = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 2),
            nn.ReLU(),
            nn.Dropout(self.settings['drop_rate']),
            nn.Linear(hidden_dim * 2, vocab_size)
        )
    
    def _setup_rnn(self):
        """
        Configure and initialize the RNN layer
        """
        rnn_params = {
            'input_size': self.settings['embed_dim'],
            'hidden_size': self.settings['hidden_dim'],
            'num_layers': self.settings['num_layers'],
            'dropout': self.settings['drop_rate'],
            'batch_first': True
        }
        
        if self.settings['rnn_type'] == 'lstm':
            self.rnn = nn.LSTM(**rnn_params)
        elif self.settings['rnn_type'] == 'gru':
            self.rnn = nn.GRU(**rnn_params)
        else:
            self.rnn = nn.RNN(**rnn_params)
    
    def _process_hidden(self, hidden):
        """
        Process hidden state based on RNN type
        """
        if self.settings['rnn_type'] == 'lstm':
            return (hidden[0], hidden[1])
        return hidden
    
    def forward(self, x, hidden):
        """
        Process single timestep through decoder
        """
        # Add sequence dimension
        x = x.unsqueeze(1)
        
        # Embed and normalize
        embedded = self.token_embedding(x)
        embedded = self.dropout(embedded)
        
        # Process through RNN
        if self.settings['rnn_type'] == 'lstm':
            hidden, cell = hidden
            output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
            hidden = (hidden, cell)
        else:
            output, hidden = self.rnn(embedded, hidden)
        
        # Remove sequence dimension
        output = output.squeeze(1)
        
        # Generate predictions
        predictions = self.output_layers(output)
        
        return predictions, hidden
    
    def get_config(self):
        """
        Return decoder configuration
        """
        return self.settings.copy()
    
    def get_output_dim(self):
        """
        Return output dimension
        """
        return self.settings['vocab_size']

In [17]:
class NeuralTranslator(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        # Add attention mechanism
        self.attention = nn.Sequential(
            nn.Linear(encoder.get_output_dim() + decoder.get_output_dim(), decoder.get_output_dim()),
            nn.Tanh(),
            nn.Linear(decoder.get_output_dim(), 1)
        )
        
        # Add context vector processing
        self.context_processor = nn.Sequential(
            nn.Linear(encoder.get_output_dim() * 2, encoder.get_output_dim()),
            nn.LayerNorm(encoder.get_output_dim()),
            nn.ReLU()
        )
        
        # Add output processing
        self.output_processor = nn.Sequential(
            nn.Linear(decoder.get_output_dim() * 2, decoder.get_output_dim()),
            nn.LayerNorm(decoder.get_output_dim()),
            nn.ReLU()
        )
    
    def _compute_attention(self, decoder_hidden, encoder_outputs):
        """
        Compute attention scores between decoder hidden state and encoder outputs
        """
        # Expand decoder hidden state to match encoder outputs
        decoder_hidden = decoder_hidden[-1].unsqueeze(1).expand(-1, encoder_outputs.size(1), -1)
        
        # Concatenate and compute attention scores
        attention_input = torch.cat([decoder_hidden, encoder_outputs], dim=2)
        attention_scores = self.attention(attention_input).squeeze(-1)
        
        # Apply softmax to get attention weights
        attention_weights = F.softmax(attention_scores, dim=1)
        
        # Compute weighted context vector
        context = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        return context, attention_weights
    
    def _process_decoder_step(self, decoder_input, decoder_hidden, encoder_outputs, context):
        """
        Process a single decoder step with attention
        """
        # Get decoder output
        decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
        
        # Compute new attention context
        new_context, _ = self._compute_attention(decoder_hidden, encoder_outputs)
        
        # Combine decoder output with context
        combined = torch.cat([decoder_output, new_context], dim=1)
        processed_output = self.output_processor(combined)
        
        return processed_output, decoder_hidden
    
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        """
        Training forward pass with attention and teacher forcing
        """
        batch_size = source.size(0)
        target_len = target.size(1)
        vocab_size = self.decoder.get_output_dim()
        
        # Initialize output tensor
        outputs = torch.zeros(batch_size, target_len, vocab_size).to(self.device)
        
        # Get encoder outputs
        encoder_outputs, encoder_hidden = self.encoder(source)
        
        # Initialize decoder
        decoder_hidden = self._initialize_decoder_state((encoder_outputs, encoder_hidden))
        decoder_input = target[:, 0]
        
        # Process each timestep
        for t in range(1, target_len):
            # Compute attention context
            context, _ = self._compute_attention(decoder_hidden, encoder_outputs)
            
            # Process decoder step
            decoder_output, decoder_hidden = self._process_decoder_step(
                decoder_input, decoder_hidden, encoder_outputs, context
            )
            
            # Store output
            outputs[:, t] = decoder_output
            
            # Determine next input
            if self._should_use_teacher_forcing(teacher_forcing_ratio):
                decoder_input = target[:, t]
            else:
                decoder_input = decoder_output.argmax(1)
        
        return outputs
    
    def _initialize_decoder_state(self, encoder_outputs):
        """
        Initialize decoder state from encoder outputs
        """
        if isinstance(encoder_outputs[1], tuple):
            return encoder_outputs[1]
        return encoder_outputs[1]
    
    def _should_use_teacher_forcing(self, teacher_forcing_ratio):
        """
        Determine if teacher forcing should be used
        """
        return random.random() < teacher_forcing_ratio
    
    def inference(self, source, max_len=50):
        """
        Inference with attention mechanism
        """
        batch_size = source.size(0)
        vocab_size = self.decoder.get_output_dim()
        
        # Initialize output tensor
        outputs = torch.zeros(batch_size, max_len, vocab_size).to(self.device)
        
        # Get encoder outputs
        encoder_outputs, encoder_hidden = self.encoder(source)
        
        # Initialize decoder
        decoder_hidden = self._initialize_decoder_state((encoder_outputs, encoder_hidden))
        decoder_input = torch.tensor([self.config['sos_token']] * batch_size, device=self.device)
        
        # Process each timestep
        for t in range(1, max_len):
            # Compute attention context
            context, _ = self._compute_attention(decoder_hidden, encoder_outputs)
            
            # Process decoder step
            decoder_output, decoder_hidden = self._process_decoder_step(
                decoder_input, decoder_hidden, encoder_outputs, context
            )
            
            # Store output
            outputs[:, t] = decoder_output
            
            # Update input
            decoder_input = decoder_output.argmax(1)
            
            # Check for end of sequence
            if (decoder_input == self.config['eos_token']).all():
                break
        
        return outputs

## Training and Evaluation of Vanilla Model

In [18]:
def train(model, train_loader, optimizer, criterion, clip=1.0, teacher_forcing_ratio=0.5):
    model.train()
    epoch_loss = 0

    for i, (src, tgt) in enumerate(train_loader):
        src = src.to(device)
        tgt = tgt.to(device)
        #print("tgt", tgt)
        optimizer.zero_grad()

        output = model(src, tgt, teacher_forcing_ratio)
        #print("output", output)
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        tgt = tgt[:, 1:].reshape(-1)
        #print("tgt2", tgt)
        #print("output2", output)
        #break

        # Calculate loss
        loss = criterion(output, tgt)

        # Backpropagation
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # Update parameters
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(train_loader)

In [19]:
def evaluate(model, val_loader, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for i, (src, tgt) in enumerate(val_loader):
            src = src.to(device)
            tgt = tgt.to(device)

            output = model.inference(src, tgt.shape[1])

            # Reshape output and target for loss calculation
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            tgt = tgt[:, 1:].reshape(-1)

            # Calculate loss
            loss = criterion(output, tgt)
            epoch_loss += loss.item()

    return epoch_loss / len(val_loader)

In [20]:
def transliterate(model, src_text, src_vocab, tgt_vocab, device, max_length=100):
    model.eval()

    # Convert source text to tensor
    src_indices = src_vocab.encode(src_text)
    src_tensor = torch.tensor(src_indices, dtype=torch.long).unsqueeze(0).to(device)

    # Get encoder outputs
    with torch.no_grad():
        if model.encoder.cell_type == 'LSTM':
            encoder_outputs, (hidden, cell) = model.encoder(src_tensor)
            decoder_hidden = (hidden, cell)
        else:
            encoder_outputs, hidden = model.encoder(src_tensor)
            decoder_hidden = hidden

    # Start with SOS token
    decoder_input = torch.tensor([tgt_vocab.char2idx[tgt_vocab.sos_token]], device=device)

    result_indices = [tgt_vocab.char2idx[tgt_vocab.sos_token]]

    for _ in range(max_length):
        with torch.no_grad():
            decoder_output, decoder_hidden = model.decoder(decoder_input, decoder_hidden)

        # Get the most likely next character
        top_token = decoder_output.argmax(1).item()
        result_indices.append(top_token)

        # Stop if EOS token
        if top_token == tgt_vocab.char2idx[tgt_vocab.eos_token]:
            break

        # Use predicted token as next input
        decoder_input = torch.tensor([top_token], device=device)

    # Convert indices to text
    result_text = tgt_vocab.decode(result_indices, remove_special_tokens=True)

    return result_text

In [21]:
# Accuracy calculation function
def calculate_accuracy(model, data_loader, src_vocab, tgt_vocab, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for src, tgt in data_loader:
            src = src.to(device)
            tgt = tgt.to(device)

            batch_size = src.shape[0]

            for i in range(batch_size):
                # Get source text and actual target text
                src_indices = src[i].tolist()
                src_text = src_vocab.decode(src_indices)
                actual_tgt_text = tgt_vocab.decode(tgt[i].tolist())

                # Get predicted transliteration
                predicted_tgt_text = transliterate(model, src_text, src_vocab, tgt_vocab, device)

                # Check if prediction matches
                if predicted_tgt_text == actual_tgt_text:
                    correct += 1
                total += 1

    return correct / total

## Sample Run of Vanilla Model

## Hyperparameter Tuning of Vanilla Model

In [24]:
def sweep_hyperparameters(config=None):
    with wandb.init(config=config):
        config = wandb.config
        wandb.run.name = f"embedding_size_{str(config.embedding_size)}_num_layers_{str(config.num_layers)}_hidden_size_{config.hidden_size} \
                          _cell_type_{config.cell_type}_dp_{config.dropout}_lr_{config.learning_rate}_batch_{config.batch_size}"

        # Log in my details
        wandb.config.update({"NAME": "KILAPARTHI VISHNU VARDHAN", "ROLL NO.": "CS24M022"})

        train_dataset = TransliterationDataset(TRAIN_FilePath, src_vocab, tgt_vocab)
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, collate_fn=collate_fn)

        dev_dataset = TransliterationDataset(DEV_FilePath, src_vocab, tgt_vocab)
        dev_loader = DataLoader(dev_dataset, batch_size=config.batch_size, shuffle=False, collate_fn=collate_fn)

        INPUT_SIZE = src_vocab.vocab_size
        OUTPUT_SIZE = tgt_vocab.vocab_size

        EMBEDDING_SIZE = config.embedding_size
        HIDDEN_SIZE = config.hidden_size
        NUM_LAYERS = config.num_layers
        CELL_TYPE = config.cell_type
        DROPOUT = config.dropout
        LEARNING_RATE = config.learning_rate
        NUM_EPOCHS = 10

        # Initialize encoder, decoder, and seq2seq model
        encoder = Encoder(
            input_size=INPUT_SIZE,
            embedding_size=EMBEDDING_SIZE,
            hidden_size=HIDDEN_SIZE,
            num_layers=NUM_LAYERS,
            cell_type=CELL_TYPE,
            dropout=DROPOUT
        )

        decoder = Decoder(
            output_size=OUTPUT_SIZE,
            embedding_size=EMBEDDING_SIZE,
            hidden_size=HIDDEN_SIZE,
            num_layers=NUM_LAYERS,
            cell_type=CELL_TYPE,
            dropout=DROPOUT
        )

        model = Seq2Seq(encoder, decoder, device).to(device)
        criterion = nn.CrossEntropyLoss(ignore_index=0)
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

        for epoch in tqdm(range(NUM_EPOCHS)):
            # Train model
            train_loss = train(model, train_loader, optimizer, criterion)
            # Evaluate model
            valid_loss = evaluate(model, dev_loader, criterion)

            print(f"Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f}")
            # Log the evaluation metrics
            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss,
                "validation_loss": valid_loss,
                })

        # Calculate accuracy on validation set
        val_accuracy = calculate_accuracy(model, dev_loader, src_vocab, tgt_vocab, device)
        print(f"\nValidation Accuracy: {val_accuracy:.4f}")
        wandb.log({
            "val_accuracy": val_accuracy
        })

In [25]:
sweep_config = {
    "method" : "bayes",
    "metric" : {"name": "val_accuracy", "goal": "maximize"},
    "parameters" : {
        "embedding_size" : {"values" : [16, 32, 64, 256]},
        "num_layers" : {"values" : [2, 3]},
        "hidden_size" : {"values" : [32, 128, 512]},
        "cell_type" : {"values" : ["RNN", "LSTM", "GRU"]},
        "dropout" : {"values" : [ 0.2, 0.3]},
        "learning_rate" : {"values" : [0.001, 0.0005]},
        "batch_size": {"values": [32, 64, 128]}
    }
}

sweep_id = wandb.sweep(sweep_config, project = "cs24m022_DA6401_Assignment3")

Create sweep with ID: rxo139kw
Sweep URL: https://wandb.ai/cs24m022-iit-madras-foundation/cs24m022_DA6401_Assignment3/sweeps/rxo139kw


In [26]:
wandb.agent('rxo139kw', function = sweep_hyperparameters, count = 30)

wandb: Agent Starting Run: ge8sbo9x with config:
wandb: 	batch_size: 64
wandb: 	cell_type: GRU
wandb: 	dropout: 0.3
wandb: 	embedding_size: 16
wandb: 	hidden_size: 512
wandb: 	learning_rate: 0.0005
wandb: 	num_layers: 2


 10%|█         | 1/10 [00:20<03:06, 20.71s/it]

Train Loss: 1.5995 | Valid Loss: 1.3383


 20%|██        | 2/10 [00:40<02:41, 20.17s/it]

Train Loss: 0.8974 | Valid Loss: 0.9699


...

Train Loss: 0.1461 | Valid Loss: 0.8616


100%|██████████| 10/10 [02:30<00:00, 15.07s/it]

Train Loss: 0.1365 | Valid Loss: 0.8908



Validation Accuracy: 0.3646


epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▃▃▂▂▁▁▁▁▁
val_accuracy,▁
validation_loss,█▅▂▁▁▂▂▃▆█
epoch,9
train_loss,0.13648
val_accuracy,0.36462
validation_loss,0.89084


## Evaluate Best Vanilla Model on Test Dataset

In [27]:
INPUT_SIZE = src_vocab.vocab_size
OUTPUT_SIZE = tgt_vocab.vocab_size
EMBEDDING_SIZE = 256
HIDDEN_SIZE = 512
NUM_LAYERS = 3
CELL_TYPE = "GRU"
DROPOUT = 0.2
LEARNING_RATE = 0.0005
BATCH_SIZE = 64
NUM_EPOCHS = 10


train_dataset = TransliterationDataset(TRAIN_FilePath, src_vocab, tgt_vocab)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

dev_dataset = TransliterationDataset(DEV_FilePath, src_vocab, tgt_vocab)
dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

test_dataset = TransliterationDataset(TEST_FilePath, src_vocab, tgt_vocab)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# Initialize encoder, decoder, and seq2seq model
encoder = Encoder(
    input_size=INPUT_SIZE,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    cell_type=CELL_TYPE,
    dropout=DROPOUT
)

decoder = Decoder(
    output_size=OUTPUT_SIZE,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    cell_type=CELL_TYPE,
    dropout=DROPOUT
)

model = Seq2Seq(encoder, decoder, device).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in tqdm(range(NUM_EPOCHS)):
    # Train model
    train_loss = train(model, train_loader, optimizer, criterion)
    # Evaluate model
    valid_loss = evaluate(model, dev_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f}")

 10%|█         | 1/10 [00:24<03:38, 24.26s/it]

Train Loss: 1.5922 | Valid Loss: 1.1444


 20%|██        | 2/10 [00:48<03:15, 24.42s/it]

Train Loss: 0.7158 | Valid Loss: 1.0068


 30%|███       | 3/10 [01:13<02:51, 24.43s/it]

...

Train Loss: 0.2321 | Valid Loss: 1.1479


 90%|█████████ | 9/10 [03:45<00:25, 25.60s/it]

Train Loss: 0.2074 | Valid Loss: 1.1636


100%|██████████| 10/10 [04:11<00:00, 25.14s/it]

Train Loss: 0.1855 | Valid Loss: 1.2250


In [28]:
train_accuracy = calculate_accuracy(model, train_loader, src_vocab, tgt_vocab, device)
val_accuracy = calculate_accuracy(model, dev_loader, src_vocab, tgt_vocab, device)
test_accuracy = calculate_accuracy(model, test_loader, src_vocab, tgt_vocab, device)

print(f"Train Accuracy : {train_accuracy*100:6.2f}%")
print(f"Val Accuracy   : {val_accuracy*100:6.2f}%")
print(f"Test Accuracy  : {test_accuracy*100:6.2f}%")

Train Accuracy :  83.37%
Val Accuracy   :  38.96%
Test Accuracy  :  39.27%


In [15]:
model.eval()
latin = []
correct_native = []
predicted_native = []

with torch.no_grad():
    for src, tgt in test_loader:
        src = src.to(device)
        tgt = tgt.to(device)

        batch_size = src.shape[0]

        for i in range(batch_size):
            # Get source text and actual target text
            src_indices = src[i].tolist()
            src_text = src_vocab.decode(src_indices)
            actual_tgt_text = tgt_vocab.decode(tgt[i].tolist())

            # Get predicted transliteration
            predicted_tgt_text = transliterate(model, src_text, src_vocab, tgt_vocab, device)

            latin.append(src_text)
            correct_native.append(actual_tgt_text)
            predicted_native.append(predicted_tgt_text)

os.makedirs("predictions_vanilla", exist_ok=True)
df = pd.DataFrame({
    "Latin": latin,
    "Correct Native": correct_native,
    "Predicted Native": predicted_native
})
df["Correct"] = df["Correct Native"] == df["Predicted Native"]

excel_path = "predictions_vanilla/test_predictions.xlsx"
df.to_excel(excel_path, index=False)

wandb.init(project="cs24m022_da6401_assignment3", name="vanilla_seq2seq_run")

samples = df.sample(n=10)
wandb_table = wandb.Table(columns=["Latin", "Correct Native", "Predicted Native", "Correct"])

for _, row in samples.iterrows():
    wandb_table.add_data(row["Latin"], row["Correct Native"], row["Predicted Native"], str(row["Correct"]))

wandb.log({"Random Test Predictions Sample": wandb_table})


NameError: name 'model' is not defined

## SEQ2SEQ Model with Attention

In [14]:
class AttentionMechanism(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        
        # Attention layers
        self.query_transform = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim)
        )
        
        self.key_transform = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim)
        )
        
        self.energy_transform = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )
    
    def _extract_hidden_state(self, hidden):
        """
        Extract the last hidden state
        """
        if isinstance(hidden, tuple):
            return hidden[0][-1]
        return hidden[-1]
    
    def _compute_attention_scores(self, query, keys):
        """
        Compute attention scores using additive attention
        """
        # Transform query and keys
        query = self.query_transform(query)
        keys = self.key_transform(keys)
        
        # Compute attention scores
        query = query.unsqueeze(1)
        combined = torch.tanh(query + keys)
        scores = self.energy_transform(combined)
        
        return scores.squeeze(-1)
    
    def _compute_context_vector(self, attention_weights, values):
        """
        Compute weighted context vector
        """
        return torch.bmm(
            attention_weights.unsqueeze(1), 
            values
        ).squeeze(1)
    
    def forward(self, hidden, encoder_outputs):
        """
        Compute attention weights and context vector
        """
        # Get last hidden state
        query = self._extract_hidden_state(hidden)
        
        # Compute attention scores
        attention_scores = self._compute_attention_scores(query, encoder_outputs)
        
        # Apply softmax to get attention weights
        attention_weights = F.softmax(attention_scores, dim=1)
        
        # Compute context vector
        context = self._compute_context_vector(attention_weights, encoder_outputs)
        
        return context, attention_weights

In [15]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()

        # Attention parameters for: e_jt = Vatt^T tanh(Uatt * s_t-1 + Watt * c_j)
        self.Uatt = nn.Linear(hidden_size, hidden_size)
        self.Watt = nn.Linear(hidden_size, hidden_size)
        self.Vatt = nn.Linear(hidden_size, 1)

    def forward(self, hidden, encoder_outputs):

        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]

        if isinstance(hidden, tuple):  # LSTM
            hidden_state = hidden[0]
            hidden_state = hidden_state[-1]
        else:
            hidden_state = hidden[-1]

        hidden_expanded = hidden_state.unsqueeze(1)

        # Uatt * s_t-1
        uatt_term = self.Uatt(hidden_expanded)

        # Watt * c_j
        watt_term = self.Watt(encoder_outputs)


        combined = torch.tanh(uatt_term + watt_term)

        # Apply Vatt^T
        energy = self.Vatt(combined)
        energy = energy.squeeze(2)

        attention_weights = F.softmax(energy, dim=1)

        # Create context vector
        context = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs)
        context = context.squeeze(1)

        return context, attention_weights

In [16]:
class AttentionDecoder(nn.Module):
    def __init__(self, output_size, embedding_size, hidden_size, attention, num_layers=1, cell_type="RNN", dropout=0.0):
        super().__init__()

        if num_layers == 1:
            dropout = 0

        self.output_size = output_size
        self.cell_type = cell_type
        self.attention = attention
        self.hidden_size = hidden_size

        # Embedding Layer
        self.embedding = nn.Embedding(output_size, embedding_size)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

        if cell_type == "LSTM":
            self.recurrent_layer = nn.LSTM(embedding_size + hidden_size, hidden_size, num_layers,
                                         dropout=dropout, batch_first=True)
        elif cell_type == "GRU":
            self.recurrent_layer = nn.GRU(embedding_size + hidden_size, hidden_size, num_layers,
                                        dropout=dropout, batch_first=True)
        else: # Default (RNN)
            self.recurrent_layer = nn.RNN(embedding_size + hidden_size, hidden_size, num_layers,
                                        dropout=dropout, batch_first=True)

        # Output layer
        self.fc_out = nn.Linear(hidden_size * 2, output_size)

    def forward(self, input, hidden, encoder_outputs):

        input = input.unsqueeze(1)
        embeddings = self.embedding(input)
        embeddings = self.dropout(embeddings)

        # Calculate attention context vector
        context, attention_weights = self.attention(hidden, encoder_outputs)

        # Concatenate embeddings and context vector
        context = context.unsqueeze(1)
        rnn_input = torch.cat((embeddings, context), dim=2)

        if self.cell_type == "LSTM":
            outputs, (hidden, cell) = self.recurrent_layer(rnn_input, hidden)
            hidden_state = hidden
            hidden_tuple = (hidden, cell)
        else:
            outputs, hidden = self.recurrent_layer(rnn_input, hidden)
            hidden_state = hidden
            hidden_tuple = hidden

        if isinstance(hidden_state, tuple):  # LSTM
            last_hidden = hidden_state[0][-1]
        else:
            last_hidden = hidden_state[-1]

        last_hidden = last_hidden.squeeze(0) if last_hidden.dim() > 2 else last_hidden

        outputs = outputs.squeeze(1)
        context = context.squeeze(1)

        output_vector = torch.cat((outputs, context), dim=1)
        prediction = self.fc_out(output_vector)

        return prediction, hidden_tuple, attention_weights

In [17]:
class NeuralTranslator(nn.Module):
    def __init__(self, encoder, decoder, attention, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.attention = attention
        self.device = device
        
        # Model configuration
        self.config = {
            'sos_token': 1,
            'eos_token': 2,
            'max_length': 50
        }
    
    def _initialize_decoder_state(self, encoder_outputs):
        """
        Initialize decoder state from encoder outputs
        """
        if isinstance(encoder_outputs[1], tuple):
            return encoder_outputs[1]
        return encoder_outputs[1]
    
    def _process_decoder_step(self, decoder_input, decoder_hidden, encoder_outputs):
        """
        Process a single decoder step
        """
        return self.decoder(decoder_input, decoder_hidden, encoder_outputs)
    
    def _should_use_teacher_forcing(self, teacher_forcing_ratio):
        """
        Determine if teacher forcing should be used
        """
        return random.random() < teacher_forcing_ratio
    
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        """
        Training forward pass
        """
        batch_size = source.size(0)
        target_len = target.size(1)
        vocab_size = self.decoder.output_size
        
        # Initialize output tensors
        outputs = torch.zeros(batch_size, target_len, vocab_size).to(self.device)
        attention_weights = torch.zeros(batch_size, target_len, source.size(1)).to(self.device)
        
        # Encode source sequence
        encoder_outputs, encoder_state = self.encoder(source)
        decoder_hidden = self._initialize_decoder_state((encoder_outputs, encoder_state))
        
        # Initialize decoder input
        decoder_input = target[:, 0]
        
        # Generate target sequence
        for t in range(1, target_len):
            # Process decoder step
            decoder_output, decoder_hidden, attention = self._process_decoder_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            
            # Store outputs
            outputs[:, t] = decoder_output
            attention_weights[:, t] = attention
            
            # Determine next input
            if self._should_use_teacher_forcing(teacher_forcing_ratio):
                decoder_input = targ

## Training and Evaluation of Attention Seq2Seq Model

In [18]:
def train_model_epoch(model, data_loader, optimizer, loss_fn, max_grad_norm=1.0, teacher_ratio=0.5):
    """
    Train model for one epoch
    """
    model.train()
    total_loss = 0
    
    for batch_idx, (source_batch, target_batch) in enumerate(data_loader):
        # Move data to device
        source_batch = source_batch.to(device)
        target_batch = target_batch.to(device)
        
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward pass
        predictions, _ = model(
            source_batch, 
            target_batch, 
            teacher_ratio
        )
        
        # Prepare tensors for loss calculation
        batch_size, seq_len, vocab_size = predictions.shape
        predictions = predictions[:, 1:].contiguous().view(-1, vocab_size)
        targets = target_batch[:, 1:].contiguous().view(-1)
        
        # Calculate loss
        batch_loss = loss_fn(predictions, targets)
        
        # Backward pass
        batch_loss.backward()
        
        # Clip gradients
        torch.nn.utils.clip_grad_norm_(
            model.parameters(), 
            max_grad_norm
        )
        
        # Update parameters
        optimizer.step()
        
        # Accumulate loss
        total_loss += batch_loss.item()
    
    # Return average loss
    return total_loss / len(data_loader)

In [19]:
def evaluate_attention_model(model, val_loader, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for i, (src, tgt) in enumerate(val_loader):
            src = src.to(device)
            tgt = tgt.to(device)

            output, _ = model.inference(src, tgt.shape[1])

            # Reshape output and target for loss calculation
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            tgt = tgt[:, 1:].reshape(-1)

            # Calculate loss
            loss = criterion(output, tgt)
            epoch_loss += loss.item()

    return epoch_loss / len(val_loader)

In [20]:
def convert_sequence(model, input_text, source_vocab, target_vocab, device, max_steps=100):
    """
    Convert sequence from source to target script
    """
    model.eval()
    
    # Process input text
    input_tokens = source_vocab.encode(input_text)
    input_tensor = torch.tensor([input_tokens], device=device)
    
    # Get encoder outputs
    with torch.no_grad():
        encoder_outputs, encoder_state = model.encoder(input_tensor)
        decoder_state = encoder_state
    
    # Initialize generation
    current_token = torch.tensor([[target_vocab.sos_token]], device=device)
    output_tokens = [target_vocab.sos_token]
    
    # Generate output sequence
    for _ in range(max_steps):
        with torch.no_grad():
            # Get decoder output
            decoder_output, decoder_state, _ = model.decoder(
                current_token, 
                decoder_state, 
                encoder_outputs
            )
            
            # Get next token
            next_token = decoder_output.argmax(1).item()
            output_tokens.append(next_token)
            
            # Check for end token
            if next_token == target_vocab.eos_token:
                break
            
            # Update current token
            current_token = torch.tensor([[next_token]], device=device)
    
    # Convert tokens to text
    return target_vocab.decode(output_tokens, remove_special_tokens=True)

In [21]:
def evaluate_translation_accuracy(model, data_loader, source_vocab, target_vocab, device):
    """
    Evaluate translation accuracy
    """
    model.eval()
    matches = 0
    samples = 0
    
    with torch.no_grad():
        for source_batch, target_batch in data_loader:
            # Move batches to device
            source_batch = source_batch.to(device)
            target_batch = target_batch.to(device)
            
            # Process each sample in batch
            for source_seq, target_seq in zip(source_batch, target_batch):
                # Decode sequences
                source_text = source_vocab.decode(source_seq.tolist())
                target_text = target_vocab.decode(target_seq.tolist())
                
                # Get model prediction
                predicted_text = translate_sequence(
                    model, 
                    source_text, 
                    source_vocab, 
                    target_vocab, 
                    device
                )
                
                # Compare prediction with target
                if predicted_text == target_text:
                    matches += 1
                samples += 1
    
    return matches / samples

def translate_sequence(model, source_text, source_vocab, target_vocab, device):
    """
    Translate a single sequence
    """
    # Encode source text
    source_tokens = source_vocab.encode(source_text)
    source_tensor = torch.tensor([source_tokens], device=device)
    
    # Get encoder outputs
    encoder_outputs, encoder_state = model.encoder(source_tensor)
    decoder_state = encoder_state
    
    # Initialize translation
    current_token = torch.tensor([[target_vocab.sos_token]], device=device)
    translated_tokens = []
    
    # Generate translation
    for _ in range(100):  # Max length limit
        # Get decoder output
        decoder_output, decoder_state, _ = model.decoder(
            current_token, 
            decoder_state, 
            encoder_outputs
        )
        
        # Get next token
        next_token = decoder_output.argmax(1).item()
        translated_tokens.append(next_token)
        
        # Check for end token
        if next_token == target_vocab.eos_token:
            break
        
        # Update current token
        current_token = torch.tensor([[next_token]], device=device)
    
    # Decode translation
    return target_vocab.decode(translated_tokens)

## Hyperparameter Sweep of Attention Seq2Seq Model

In [22]:
def optimize_model_parameters(config=None):
    """
    Optimize model parameters using wandb sweeps
    """
    with wandb.init(config=config) as run:
        # Set run name based on parameters
        run_name = f"model_opt_emb{config.embedding_size}_layers{config.num_layers}_hidden{config.hidden_size}_type{config.cell_type}_drop{config.dropout}_lr{config.learning_rate}_batch{config.batch_size}"
        wandb.run.name = run_name
        
        # Add user information
        wandb.config.update({
            "student_name": "KILAPARTHI VISHNU VARDHAN",
            "student_id": "CS24M022"
        })
        
        # Create data loaders
        train_data = TransliterationDataset(TRAIN_FilePath, src_vocab, tgt_vocab)
        train_loader = DataLoader(
            train_data, 
            batch_size=config.batch_size, 
            shuffle=True, 
            collate_fn=collate_fn
        )
        
        val_data = TransliterationDataset(DEV_FilePath, src_vocab, tgt_vocab)
        val_loader = DataLoader(
            val_data, 
            batch_size=config.batch_size, 
            shuffle=False, 
            collate_fn=collate_fn
        )
        
        # Model configuration
        model_config = {
            'input_dim': src_vocab.vocab_size,
            'output_dim': tgt_vocab.vocab_size,
            'embed_dim': config.embedding_size,
            'hidden_dim': config.hidden_size,
            'num_layers': config.num_layers,
            'rnn_type': config.cell_type,
            'drop_rate': config.dropout
        }
        
        # Initialize model components
        encoder = SequenceEncoder(**model_config)
        attention = AttentionMechanism(model_config['hidden_dim'])
        decoder = SequenceDecoder(
            **model_config,
            attention=attention
        )
        
        # Create and move model to device
        model = NeuralTranslator(encoder, decoder, attention, device).to(device)
        
        # Setup training
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
        
        # Training loop
        for epoch in tqdm(range(10)):
            # Training phase
            train_metrics = train_model(
                model, 
                train_loader, 
                optimizer, 
                loss_fn
            )
            
            # Validation phase
            val_metrics = evaluate_model(
                model, 
                val_loader, 
                loss_fn
            )
            
            # Log metrics
            metrics = {
                'epoch': epoch,
                'train_loss': train_metrics['loss'],
                'val_loss': val_metrics['loss']
            }
            wandb.log(metrics)
            
            print(f"Epoch {epoch}: Train Loss = {metrics['train_loss']:.4f}, Val Loss = {metrics['val_loss']:.4f}")
        
        # Final evaluation
        accuracy = compute_accuracy(
            model, 
            val_loader, 
            src_vocab, 
            tgt_vocab, 
            device
        )
        
        print(f"\nFinal Validation Accuracy: {accuracy:.4f}")
        wandb.log({'final_accuracy': accuracy})

In [ ]:
sweep_config = {
    "method" : "bayes",
    "metric" : {"name": "val_accuracy", "goal": "maximize"},
    "parameters" : {
        "embedding_size" : {"values" : [16, 32, 64, 256]},
        "num_layers" : {"values" : [2, 3]},
        "hidden_size" : {"values" : [32, 128, 512]},
        "cell_type" : {"values" : ["RNN", "LSTM", "GRU"]},
        "dropout" : {"values" : [ 0.2, 0.3]},
        "learning_rate" : {"values" : [0.001, 0.0005]},
        "batch_size": {"values": [32, 64, 128]}
    }
}

sweep_id = wandb.sweep(sweep_config, project = "cs24m022_DA6401_Assignment3")

Create sweep with ID: tk2okbfr
Sweep URL: https://wandb.ai/cs24m022-iit-madras-foundation/cs24m022_DA6401_Assignment3/sweeps/tk2okbfr


In [24]:
sweep_id = 'ukpef1o2'
wandb.agent('ukpef1o2', function = sweep_hyperparameters, count = 40)

wandb: Agent Starting Run: nc08qgtb with config:
wandb: 	batch_size: 64
wandb: 	cell_type: RNN
wandb: 	dropout: 0.2
wandb: 	embedding_size: 256
wandb: 	hidden_size: 512
wandb: 	learning_rate: 0.0005
wandb: 	num_layers: 2
wandb: Ctrl + C detected. Stopping sweep.


In [29]:
def extract_attention_patterns(model, input_text, source_vocab, target_vocab, device, max_steps=100):
    """
    Extract attention patterns during translation
    """
    model.eval()
    
    # Convert input text to tensor
    input_tokens = source_vocab.encode(input_text)
    input_tensor = torch.tensor([input_tokens], device=device)
    
    # Get encoder outputs
    with torch.no_grad():
        encoder_outputs, encoder_state = model.encoder(input_tensor)
        decoder_state = encoder_state
    
    # Initialize tracking
    attention_patterns = []
    output_tokens = []
    
    # Start with start token
    current_token = torch.tensor([[target_vocab.sos_token]], device=device)
    
    # Generate translation
    for _ in range(max_steps):
        with torch.no_grad():
            # Get decoder output and attention
            decoder_output, decoder_state, attention = model.decoder(
                current_token, decoder_state, encoder_outputs
            )
            
            # Store attention pattern
            attention_patterns.append(attention.squeeze().cpu().numpy())
            
            # Get next token
            next_token = decoder_output.argmax(1).item()
            output_tokens.append(next_token)
            
            # Check for end token
            if next_token == target_vocab.eos_token:
                break
            
            # Update current token
            current_token = torch.tensor([[next_token]], device=device)
    
    return attention_patterns, output_tokens

In [30]:
def generate_attention_visualization(model, data_loader, source_vocab, target_vocab, device, num_examples=10):
    """
    Generate attention visualization with a completely different approach
    """
    model.eval()
    
    # Create a custom figure layout with subplots
    fig = plt.figure(figsize=(20, 5 * num_examples))
    gs = gridspec.GridSpec(num_examples, 2, width_ratios=[1, 2])
    
    # Custom color scheme for attention
    attention_cmap = plt.cm.viridis
    
    # Process data in batches
    translation_pairs = []
    for batch in data_loader:
        source_batch, target_batch = batch
        for s, t in zip(source_batch, target_batch):
            translation_pairs.append((s, t))
    
    # Select random examples
    selected_pairs = random.sample(translation_pairs, min(num_examples, len(translation_pairs)))
    
    for idx, (source_seq, target_seq) in enumerate(selected_pairs):
        # Decode sequences
        source_text = source_vocab.decode(source_seq.tolist(), remove_special_tokens=True)
        target_text = target_vocab.decode(target_seq.tolist(), remove_special_tokens=True)
        
        # Get attention patterns
        attention_patterns, predicted_tokens = analyze_attention_patterns(
            model, source_text, source_vocab, target_vocab, device
        )
        
        # Create subplots for each example
        ax1 = plt.subplot(gs[idx, 0])  # Text visualization
        ax2 = plt.subplot(gs[idx, 1])  # Attention heatmap
        
        # Visualize text alignment
        visualize_text_alignment(
            ax1, 
            source_text, 
            target_text, 
            predicted_tokens, 
            target_vocab
        )
        
        # Visualize attention heatmap
        visualize_attention_heatmap(
            ax2,
            attention_patterns,
            source_text,
            predicted_tokens,
            source_vocab,
            target_vocab,
            attention_cmap
        )
    
    plt.tight_layout()
    
    # Log to wandb with custom formatting
    wandb.init(project="translation_visualization", name="attention_patterns")
    wandb.log({
        "attention_visualization": wandb.Image(fig),
        "visualization_type": "translation_attention",
        "num_examples": num_examples
    })
    
    plt.show()

def analyze_attention_patterns(model, source_text, source_vocab, target_vocab, device):
    """
    Analyze attention patterns during translation
    """
    # Encode source text
    source_indices = source_vocab.encode(source_text)
    source_tensor = torch.tensor([source_indices], device=device)
    
    # Initialize attention tracking
    attention_weights = []
    predicted_tokens = []
    
    # Get encoder outputs
    with torch.no_grad():
        encoder_outputs, encoder_hidden = model.encoder(source_tensor)
        
        # Initialize decoder
        decoder_hidden = model._initialize_decoder_state((encoder_outputs, encoder_hidden))
        decoder_input = torch.tensor([[target_vocab.sos_token]], device=device)
        
        # Generate translation
        for _ in range(model.config['max_length']):
            # Get attention context
            context, attention = model._compute_attention(decoder_hidden, encoder_outputs)
            attention_weights.append(attention.squeeze().cpu().numpy())
            
            # Generate next token
            output, decoder_hidden = model._process_decoder_step(
                decoder_input, decoder_hidden, encoder_outputs, context
            )
            
            # Get predicted token
            predicted_token = output.argmax(1).item()
            predicted_tokens.append(predicted_token)
            
            # Check for end of sequence
            if predicted_token == target_vocab.eos_token:
                break
                
            decoder_input = torch.tensor([[predicted_token]], device=device)
    
    return attention_weights, predicted_tokens

def visualize_text_alignment(ax, source_text, target_text, predicted_tokens, target_vocab):
    """
    Visualize text alignment between source and target
    """
    predicted_text = target_vocab.decode(predicted_tokens, remove_special_tokens=True)
    
    # Create text visualization
    ax.text(0.5, 0.7, f"Source: {source_text}", 
            ha='center', va='center', fontproperties=hindi_font, fontsize=12)
    ax.text(0.5, 0.5, f"Target: {target_text}", 
            ha='center', va='center', fontproperties=hindi_font, fontsize=12)
    ax.text(0.5, 0.3, f"Predicted: {predicted_text}", 
            ha='center', va='center', fontproperties=hindi_font, fontsize=12)
    
    ax.axis('off')

def visualize_attention_heatmap(ax, attention_patterns, source_text, predicted_tokens, 
                              source_vocab, target_vocab, cmap):
    """
    Visualize attention heatmap with custom formatting
    """
    # Convert attention patterns to numpy array
    attention_matrix = np.array(attention_patterns)
    
    # Get character representations
    source_chars = [source_vocab.idx2char[i] for i in source_vocab.encode(source_text)]
    target_chars = [target_vocab.idx2char[i] for i in predicted_tokens]
    
    # Create heatmap
    im = ax.imshow(attention_matrix, cmap=cmap, aspect='auto')
    
    # Customize axis labels
    ax.set_xticks(np.arange(len(source_chars)))
    ax.set_yticks(np.arange(len(target_chars)))
    ax.set_xticklabels(source_chars, fontproperties=hindi_font, fontsize=10)
    ax.set_yticklabels(target_chars, fontproperties=hindi_font, fontsize=10)
    
    # Add colorbar
    plt.colorbar(im, ax=ax)
    
    # Rotate x-axis labels
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

In [ ]:
plot_attention_heatmap(model, test_loader, src_vocab, tgt_vocab, device, num_samples=10)

/tmp/ipykernel_35/3240096877.py:45: UserWarning: Glyph 8594 (\N{RIGHTWARDS ARROW}) missing from current font.
  plt.tight_layout()


/usr/local/lib/python3.11/dist-packages/wandb/sdk/data_types/image.py:307: UserWarning: Glyph 8594 (\N{RIGHTWARDS ARROW}) missing from current font.
  util.ensure_matplotlib_figure(data).savefig(buf, format=self.format)
/usr/local/lib/python3.11/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 8594 (\N{RIGHTWARDS ARROW}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)


In [ ]:
import os, json, random, torch, wandb
from IPython.display import HTML

# 1) Manual list of Latin words
sampled = ["Damn", "world", "need", "model", "example"]
print("Sampled words:", sampled)

# 2) Inline HTML generator
def create_interactive_connectivity(attn_matrix, input_seq, output_seq, filename="attention.html"):
    html_template = """<!DOCTYPE html><html><head><meta charset="UTF-8"><title>Attention</title><style>
 body{font-family:'Arial Unicode MS','Noto Sans Devanagari',sans-serif;margin:20px;text-align:center;}
 .container{display:inline-block;}
 .controls{margin:12px;}
 .slider{width:180px;}
 .output-chars,.input-chars{display:flex;justify-content:center;margin:10px;}
 .char{padding:6px 10px;margin:3px;font-size:18px;position:relative;cursor:pointer;min-width:24px;transition:all .2s;}
 .output-char{background:#f0f0f0;border-radius:4px;}
 .input-char{background:#e0e0e0;border-radius:4px;}
 .selected{background:#4caf50;color:#fff;font-weight:bold;box-shadow:0 0 6px rgba(76,175,80,.5);}
 .highlighted{background:rgba(76,175,80,.3);transform:scale(1.1);}
 .connection-line{position:absolute;background:rgba(0,200,0,.5);height:3px;transform-origin:left center;z-index:-1;pointer-events:none;}
</style></head><body>
 <div class="container">
  <div class="controls">
    Threshold:
    <input id="thr" type="range" min="0" max="100" value="30" class="slider">
    <span id="tv">0.30</span>
  </div>
  <div class="output-chars" id="outs"></div>
  <div class="input-chars" id="ins"></div>
 </div>
 <script>
  const A={attn_matrix},IN={input_seq},OUT={output_seq};
  let cur=0,thr=0.3;
  function init(){renderOut();renderIn();draw();
    document.getElementById('thr').oninput=e=>{thr=e.target.value/100;document.getElementById('tv').textContent=thr.toFixed(2);draw();};
    window.onresize=draw;
  }
  function renderOut(){let c=document.getElementById('outs');c.innerHTML='';
    OUT.forEach((ch,i)=>{let d=document.createElement('div');
      d.className=`char output-char ${i===cur?'selected':''}`;d.textContent=ch;d.dataset.i=i;
      d.onmouseover=d.onclick=()=>{cur=i;renderOut();draw();};c.appendChild(d);
    });
  }
  function renderIn(){let c=document.getElementById('ins');c.innerHTML='';
    IN.forEach((ch,i)=>{let d=document.createElement('div');
      d.className='char input-char';d.textContent=ch;d.dataset.i=i;c.appendChild(d);
    });
  }
  function draw(){
    document.querySelectorAll('.connection-line').forEach(e=>e.remove());
    document.querySelectorAll('.input-char').forEach(e=>e.classList.remove('highlighted'));
    let o=document.querySelector(`.output-char[data-i="${cur}"]`);if(!o)return;
    let R=o.getBoundingClientRect(),W=A[cur],M=Math.max(...W);
    W.forEach((w,i)=>{let n=w/M; if(n<thr) return;
      let inp=document.querySelector(`.input-char[data-i="${i}"]`);
      inp.classList.add('highlighted');let S=inp.getBoundingClientRect();
      let x1=R.left+R.width/2-window.scrollX,y1=R.top+R.height-window.scrollY,
          x2=S.left+S.width/2-window.scrollX,y2=S.top-window.scrollY;
      let L=Math.hypot(x2-x1,y2-y1),ang=Math.atan2(y2-y1,x2-x1)*180/Math.PI;
      let line=document.createElement('div');line.className='connection-line';
      Object.assign(line.style,{width:`${L}px`,left:`${x1}px`,top:`${y1}px`,transform:`rotate(${ang}deg)`,opacity:n});
      document.body.appendChild(line);
    });
  }
  document.addEventListener('DOMContentLoaded',init);
</script></body></html>"""

    # sanitize attention matrix
    clean = []
    for row in attn_matrix:
        if hasattr(row, "cpu"):
            row = row.cpu().numpy()
        clean.append([float(x) for x in row])
    attn_json = json.dumps(clean)
    inp_json  = json.dumps(input_seq)
    out_json  = json.dumps(output_seq)

    html = html_template.replace("{attn_matrix}", attn_json)\
                        .replace("{input_seq}", inp_json)\
                        .replace("{output_seq}", out_json)
    with open(filename, "w", encoding="utf-8") as f:
        f.write(html)
    return filename

# 3) Build & log the W&B table
wandb.init(project="cs24m022_da6401_assignment3", resume="allow")
table = wandb.Table(columns=["Input (Latin)", "Prediction (Devanagari)", "Attention (HTML)"])

model.eval()
for i, latin in enumerate(sampled):
    attn_weights, pred_ids = get_attention_weights(model, latin, src_vocab, tgt_vocab, device)
    pred_str = tgt_vocab.decode(pred_ids, remove_special_tokens=True)

    html_file = create_interactive_connectivity(
        attn_matrix=attn_weights,
        input_seq=list(latin),
        output_seq=list(pred_str),
        filename=f"attn_{i}.html"
    )
    html_txt = open(html_file, "r", encoding="utf-8").read()
    table.add_data(latin, pred_str, wandb.Html(html_txt))
    os.remove(html_file)

# Log & summary
wandb.log({"Interactive_Attention_Table": table})
wandb.run.summary["Interactive_Attention_Examples"] = table

# Optional display
display(HTML(html_file))


Sampled words: ['Damn', 'world', 'need', 'model', 'example']


epoch,9
train_loss,0.23786
val_accuracy,0.33869
validation_loss,0.74025
